In [2]:
import sys
sys.path.append('../src')


In [3]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import os


In [4]:
from gait.utils import compute_resultant_acceleration, split_test_train_by_subjects, compute_resultant_gyro, compute_resultant_angle, get_overlap_data_all_sessions, create_dir
from gait.training import change_x_lstm_shape


In [5]:
def change_shape_lstm(X):
    n_steps, n_length = 4, 32
    return X.reshape((X.shape[0], n_steps, n_length, 12))



In [6]:
def get_std_values(X, channel):
    return X[:, :, channel].std(axis=1)


def get_mean_values(X, channel):
    return X[:, :, channel].mean(axis=1)


def get_max_values(X, channel):
    return X[:, :, channel].max(axis=1)


def get_statistic_feature_all_channels(X):
    statistic_features = []

    for channel in range(9,12):
        value = get_std_values(X, channel)
        print('value shape', value.shape)
        statistic_features.append(value)
        statistic_features.append(get_mean_values(X, channel))
        statistic_features.append(get_max_values(X, channel))

    return np.vstack(statistic_features).T

In [7]:
OVERLAP_PERCENT = 80
# overlapPercents = [0]
exclude_subjects = ['ddAeJA42PXvwthbW', 'nan', 'sUZBISq61Y7I5tqQ', 'LLZjAPTyj7muHsEf',
                    'cbOZWnI7s1y8oLD4', 'EUbKPOSQgjccjtvi', 'MMuX9YIh4NTbLZLM',
                    'PE8D53oX060qLbdX', 'ddAeJA42PXvwthbW', 'xYdtS1F8tDyjEIgN', '19AoxD1bgrDckd2p',
                    'wtyNo4LYaWXrkzA7']

X, y, subjects = get_overlap_data_all_sessions(OVERLAP_PERCENT)
# REMOVE UNWANTED SUBJECTS
indexes = np.where(subjects == exclude_subjects)
X = np.delete(X, indexes[0], axis=0)
y = np.delete(y, indexes[0], axis=0)
subjects = np.delete(subjects, indexes[0], axis=0)
# END REMOVE UNWANTED SUBJECTS
# AUGMENT DATA
resultant_acc = compute_resultant_acceleration(X)
resultant_gyro = compute_resultant_gyro(X)
resultant_angle = compute_resultant_angle(X)
resultant_acc = resultant_acc.reshape(
    resultant_acc.shape[0], resultant_acc.shape[1], 1)
resultant_gyro = resultant_gyro.reshape(
    resultant_gyro.shape[0], resultant_gyro.shape[1], 1)
resultant_angle = resultant_angle.reshape(
    resultant_angle.shape[0], resultant_angle.shape[1], 1)
X = np.concatenate((X, resultant_acc), axis=2)
X = np.concatenate((X, resultant_gyro), axis=2)
X = np.concatenate((X, resultant_angle), axis=2)
# AUGMENT DATA END


In [9]:
# trainX = change_x_lstm_shape(X)
# print('UNIQUE subjects:', np.unique(subjects))
X_train, X_test, y_train, y_test, y_train_non_encoded, y_test_non_encoded = split_test_train_by_subjects(
    X, y, subjects, train_percent=0.9)
X_train_stats = get_statistic_feature_all_channels(X_train)
X_test_stats = get_statistic_feature_all_channels(X_test)
print(X_train_stats.shape)
representative_dataset = X.astype(np.float32)
representative_stat_dataset = X_train_stats.astype(np.float32)



value shape (69508,)
value shape (69508,)
value shape (69508,)
value shape (11024,)
value shape (11024,)
value shape (11024,)
(69508, 9)


In [10]:
import tensorflow_model_optimization as tfmot



In [11]:
with tfmot.quantization.keras.quantize_scope():
  model_path = '../models/model_80_overlap/best_model.43-0.69-0.76.hdf5'
  loaded_model = keras.models.load_model(model_path)
  loaded_model.summary()
  QUANTIZED_ROOT = './quantized'
  tflite_model_name = 'model.tflite'
  quantized_model_name = 'model_quantized.tflite'
  print(model_path.split('/')[3])
  model_paths = model_path.split('/')
  
  tflite_path = os.path.join(QUANTIZED_ROOT, *model_paths[2:-1])
  tflite_filepath = os.path.join(
      QUANTIZED_ROOT, *model_paths[2:-1], tflite_model_name)
  create_dir(tflite_path)
  
  converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
  converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
  tflite_model = converter.convert()
  open(tflite_filepath, 'wb').write(tflite_model)



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 12, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 quantize_layer (QuantizeLayer)  (None, 128, 12, 1)  3           ['input_3[0][0]']                
                                                                                                  
 quant_conv2d_2 (QuantizeWrappe  (None, 64, 6, 96)   14113       ['quantize_layer[0][0]']         
 rV2)                                                                                             
                                                                                            

INFO:tensorflow:Assets written to: /tmp/tmpzb0b8_yf/assets


INFO:tensorflow:Assets written to: /tmp/tmpzb0b8_yf/assets


In [12]:
print(X_train.shape)
print(X_train_stats.shape)

(69508, 128, 12)
(69508, 9)


In [13]:
X_train = X_train.reshape(-1, X_train.shape[1], X_train.shape[2], 1).astype(np.float32)
representative_dataset = X_train.astype(np.float32)
representative_stat_dataset = X_train_stats.astype(np.float32)


# def representative_dataset_gen():
#     for data in tf.data.Dataset.from_tensor_slices((representative_dataset)).batch(1).take(100):
#         for stat_data in tf.data.Dataset.from_tensor_slices((representative_stat_dataset)).batch(1).take(100):
#             yield [tf.dtypes.cast(data, tf.float32), tf.dtypes.cast(stat_data, tf.float32)]

def representative_dataset_gen():
    for _ in range(100):
        data1, data2 = np.random.rand(1, 128, 12, 1), np.random.rand(1, 12)
        yield [data1.astype(np.float32), data2.astype(np.float32)]


In [14]:
with tfmot.quantization.keras.quantize_scope():
  quantized_model_name = 'model_quantized.tflite'
  model_path = '../best_model/cnn_stat/80_overlap/best_model.100-0.59-0.78.hdf5'
  loaded_model = keras.models.load_model(model_path)
  converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
  converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
  tflite_model = converter.convert()
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = representative_dataset_gen
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  converter.inference_input_type = tf.int8  # or tf.uint8
  converter.inference_output_type = tf.int8  # or tf.uint8
  tflite_quantized_model = converter.convert()
  
  tflite_quantized_model_filtepath =  os.path.join(
      QUANTIZED_ROOT, *model_paths[2:-1], quantized_model_name)
  
  open(tflite_quantized_model_filtepath, "wb").write(tflite_model)

  
  # print("Basic model is %d bytes" % basic_model_size)
  # quantized_model_size = os.path.getsize(tflite_quantized_model)
  # print("Quantized model is %d bytes" % quantized_model_size)
  # difference = basic_model_size - quantized_model_size
  # print("Difference is %d bytes" % difference)

INFO:tensorflow:Assets written to: /tmp/tmpsajni_za/assets


INFO:tensorflow:Assets written to: /tmp/tmpsajni_za/assets


INFO:tensorflow:Assets written to: /tmp/tmp77t87ggf/assets


INFO:tensorflow:Assets written to: /tmp/tmp77t87ggf/assets
/home/sanjeev/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [37]:
print(
tflite_quantized_model_filtepath

)

./quantized/cnn_stat/80_overlap/model_quantized.tflite
